In [1]:
import time
import pprint
import glob
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
import string
import pickle
import gensim
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import stem_text
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath, get_tmpfile
from gensim.similarities import Similarity
from gensim.corpora.textcorpus import TextDirectoryCorpus
from nltk.tokenize import word_tokenize
from gensim.corpora import MmCorpus 
import numpy as np
import sys

In [2]:
files = glob.glob('/home/rohan/Documents/Homework/WSTA/wiki-all/*.txt')

In [3]:
def preprocess(text):
    return [word for word in word_tokenize(stem_text(strip_punctuation(text.lower().replace("\n", ""))))]

In [4]:
start = time.time()
doc_id_list = []
line_num_list = []
sent_list = []
df = pd.DataFrame(columns = ['doc_id', 'line'])
for file in files:
    with open(file, 'r') as f:
        print(file)
        for line in f:
            try:
                #print(line)
                row = line.split(maxsplit=2)
                word = row[2].split()
                if len(word) > 4:
                    doc_id_list.append(row[0]+"#"+row[1])
                    sent_list.append(row[2].rstrip('\n'))          
                else:
                    pass
            except Exception as e:
                print("ERROR",e, line)
df['doc_id'] = doc_id_list
df['line'] = sent_list
print(df.head())
print(df.shape)
print("Time taken: ", time.time() - start)

/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-047.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-041.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-107.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-108.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-098.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-030.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-031.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-087.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-077.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-080.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-010.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-045.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-066.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-052.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-040.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-069.txt
/home/rohan/Documents/Homework/WSTA/wiki-all/wiki-022.txt
/home/rohan/Do

In [8]:
start = time.time()
np.savetxt('/home/rohan/Documents/Homework/WSTA/wiki_fullid_filtered.txt', df['doc_id'].values, fmt = '%s')
np.savetxt('/home/rohan/Documents/Homework/WSTA/wiki_sent_filtered.txt', df['line'], fmt = '%s')
print("Time taken: ", time.time() - start)

Time taken:  57.22614908218384


In [9]:
start = time.time()
corpus = TextCorpus(datapath('/home/rohan/Documents/Homework/WSTA/wiki_sent_filtered.txt'))
print("Time taken: ", time.time()-start)

Time taken:  5416.398795604706


In [10]:
start = time.time()
dictionary = corpus.dictionary
dictionary.save_as_text('/home/rohan/Documents/Homework/WSTA/dictionaries/dict_filtered.txt')
print("Time taken: ", time.time()-start)

Time taken:  7.851433038711548


In [11]:
start = time.time()
tf_idf = gensim.models.TfidfModel(corpus)
tf_idf.save('/home/rohan/Documents/Homework/WSTA/tf-idfs/tf_idf_filtered')
print(time.time() - start)

1409.1308815479279


In [12]:
start = time.time()
sims = gensim.similarities.Similarity('/home/rohan/Documents/Homework/WSTA/sims/filtered',tf_idf[corpus], num_features=len(dictionary))
print("Time taken: ", time.time()-start)

Time taken:  7531.008040428162


In [24]:
sims.save('/home/rohan/Documents/Homework/WSTA/sims/sims_index_filtered')

In [23]:
query = input("Enter statement: ")
start = time.time()
query_doc = preprocess(query)
query_doc_bow = dictionary.doc2bow(query_doc)
query_doc_tf_idf = tf_idf[query_doc_bow]
sim_list = sims[query_doc_tf_idf]
df['similarity_score'] = sim_list
print(df.nlargest(50, ['similarity_score']))
print("Time taken: ",time.time() - start)

Enter statement:  where do we go now


                                               doc_id  \
0                Israeli_legislative_election,_2013#0   
1                Israeli_legislative_election,_2013#1   
2                Israeli_legislative_election,_2013#2   
3                Israeli_legislative_election,_2013#5   
4                Israeli_legislative_election,_2013#6   
5   International_Mathematics_and_Design_Associati...   
6   International_Mathematics_and_Design_Associati...   
7   International_Mathematics_and_Design_Associati...   
8                             Interstellar_Low_Ways#0   
9                             Interstellar_Low_Ways#1   
10                            Interstellar_Low_Ways#2   
11                            Interstellar_Low_Ways#4   
12                            Interstellar_Low_Ways#5   
13                            Interstellar_Low_Ways#6   
14                            Interstellar_Low_Ways#9   
15                           Interstellar_Low_Ways#12   
16                             